# 从零开始学习自然语言处理

# 1. 了解一下什么是自然语言处理
阅读一下 paddle 官网的文档: [自然语言处理：自然语言处理综述](https://www.paddlepaddle.org.cn/tutorials/projectdetail/3518060)

一个重要的信息是： 语言是一种文本信息，而从计算角度，计算机无法直接处理这些文本。计算机是计算的机器，现有的计算机都以浮点数为输入和输出，擅长执行加减乘除类计算。自然语言本身并不是浮点数，计算机为了能存储和显示自然语言，需要把自然语言中的字符转换为一个固定长度（或者变长）的二进制编码：UTF-8。

但是这个编码本身不是数字，对这个编码的计算往往不具备数学和物理含义。例如：把“法国”和“首都”放在一起，大多数人首先联想到的内容是“巴黎”。但是如果我们使用“法国”和“首都”的UTF-8编码去做加减乘除等运算，是无法轻易获取到“巴黎”的UTF-8编码，甚至无法获得一个有效的UTF-8编码。因此，如何让计算机可以有效地计算自然语言，是计算机科学家和工程师面临的巨大挑战。


# 2. 词嵌入/词向量（Word Embedding）
正如 1 中所说，如何把语言转换为“有意义的”浮点数，是一个挑战。Word Embedding 就是一种把词变成向量的方法（把词转换为N维空间的一个点），方便后续使用。

首先的一点是，“词是什么？”。或者说词是编码的最小单位。 汉字的数目是有限的，为什么不直接把每个汉字进行编码。而词的数目可能会不断的增加，不通的时代，词甚至有不一样的意思，而且我们总会创建一些新词。

比如一句话： “我爱人工智能”，如果以词来切分，就可以切分为： “我“， ”爱“， ”人工智能“

如果以每个汉字来切分，“我“， ”爱“， ”人”， “工” ，“智”， “能“。

很显然，从人类理解的角度出发，按词切分显得更合理（如果你硬要按汉字切分，当然也可以，但是古先圣贤的经验告诉我们，按词分是更好的）。

在这之前，先想一下有哪些方式呢，如果把每一个词字转换为一个 one-hot 向量，这是一种最简单且可行的方式：比如假设词有 1000 个(实际数目远远不止)，

那么第一个词对应的向量为: 1, 0, 0, 0, 0 .... 

第二个词为:             0, 1, 0, 0, 0, ...

第三个词为:             0, 0, 1, 0, 0, ...

但是存在的问题就是，词之间的联系却没有了，比如 “香蕉”，“橘子”这两个词，似乎是有一定联系的，但是这种词向量使得这 1000 个词之间的距离都是一样的。难以体现之间的联系（one hot 之间正交），因此我们可能需要在这 one-hot encoding 的基础上再做一些事情（乘以 Embedding 矩阵）。

大部分词向量模型都需要回答两个问题：

**如何把词转换为向量?**
自然语言单词是离散信号，比如“香蕉”，“橘子”，“水果”在我们看来就是3个离散的词。如何把每个离散的单词转换为一个向量？

**如何让向量具有语义信息?**
比如，我们知道在很多情况下，“香蕉”和“橘子”更加相似，而“香蕉”和“句子”就没有那么相似，同时“香蕉”和“食物”、“水果”的相似程度可能介于“橘子”和“句子”之间。

所以一个好的 Word Embedding 意义重大，在很大程度上决定了 nlp 任务的成败。

阅读 paddle 官网文档： [自然语言处理：词向量 Word Embedding](https://www.paddlepaddle.org.cn/tutorials/projectdetail/3578658)

可以在线执行官网的代码，或者在自己的环境中执行，参考 [Word Embeding](./Word%20Embeding.ipynb)

这里面介绍了使用 CBOW 和 Skip-gram 的算法训练得到了一个词向量模型。如有兴趣可以跟着流程走一遍，实际上我们只需要使用古先圣贤们训练好的词向量模型就行。


In [ ]:
'''
使用 PaddleNLP 已预置多个公开的预训练的中文 Embedding
注意这里使用的是 TokenEmbedding 而不是上面一直说的 WordEmbedding 

Token 是更专业的术语，Token可以是 Word（词），也可以是单个的字。nlp任务里的 Token 一般都是 Word，故在后面的描述中可以认为 Token == Word 
'''

from paddlenlp.embeddings import TokenEmbedding
import paddle

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

# 查看token_embedding详情
print(token_embedding)

'''
可以看出，token_embedding 的shape 是[635965, 300]，即一共有 635965 个词，包括UNK，每个词由 300 纬的向量组成。

从源码实现上来看， 继承关系为： TokenEmbedding --> nn.Embedding --> nn.Layer；是paddle中一个标准的 layer。
'''


In [ ]:

'''
TokenEmbedding 暴露的主要 API 为：
search， get_idx_from_word， get_idx_list_from_words，cosine_sim
'''

# 使用 search() 获得指定词汇的词向量。
embedding_tensor = token_embedding.search("中国")
# print(embedding_tensor)

# 也可以通过这种方式：先获取 token 对应的 idx，再根据 idx 获得 token_embedding
idx = token_embedding.get_idx_from_word("中国")
embedding_tensor = token_embedding(paddle.to_tensor(idx))
print(type(embedding_tensor))
print(embedding_tensor.shape)
# print(embedding_tensor)

# embedding 多个词向量
idxs = []
idxs.append(token_embedding.get_idx_from_word("中国"))
idxs.append(token_embedding.get_idx_from_word("北京"))
embedding_tensors = token_embedding(paddle.to_tensor(idxs))
print(type(embedding_tensors))
print(embedding_tensors.shape)


# cosine_sim() 计算词向量间余弦相似度，语义相近的词语余弦相似度更高，说明预训练好的词向量空间有很好的语义表示能力
score1 = token_embedding.cosine_sim("女孩", "女人")
score2 = token_embedding.cosine_sim("女孩", "书籍")
print('score1:', score1)
print('score2:', score2)

# 3. 句向量（Sentence Embedding）
有了词向量，如何得到句向量呢？

此时我们可以使用词袋模型（Bag of Words，简称BoW）计算句子的语义向量。

**首先**，将两个句子分别进行切词，并在 TokenEmbedding 中查找相应的单词词向量（word embdding）。

**然后**，根据词袋模型，将句子的 word embedding 叠加作为句子向量（sentence embedding）。

切词可以使用 jieba 第三方库，参考[jieba分词工具](./jieba%E5%88%86%E8%AF%8D%E5%B7%A5%E5%85%B7.ipynb)

也可以使用 JiebaTokenizer, 这是对 jieba 的简单封装。

In [31]:
# 使用 JiebaTokenizer 进行分词
from paddlenlp.data import JiebaTokenizer
import paddlenlp

vocab = token_embedding.vocab

'''
token_embedding 的核心内容就是一个 [635965, 300]的词表（vocab）；
vocab 本质是一个 tocken_to_idx 的字典
'''

tokenizer = JiebaTokenizer(vocab) # 分词器使用提供的词表进行分词
text = "我来自清华大学"
tokens = tokenizer.cut(text)
print(tokens)

token_idxs = token_embedding.get_idx_list_from_words(tokens)
embedding_tensors = token_embedding(paddle.to_tensor(token_idxs))
print(embedding_tensors)

# 使用词袋模型得到句向量
bow_encoder = paddlenlp.seq2vec.BoWEncoder(token_embedding.embedding_dim)

# 这里修改 shape 的原因是：BoWEncoder 的 forward 函数要求输入的格式是 (batch_size, num_tokens, emb_dim)
# print(embedding_tensors.shape)
paddle.unsqueeze_(embedding_tensors, axis=0)
# print(embedding_tensors.shape)
sentense_embedding = bow_encoder(embedding_tensors)
print(sentense_embedding)
# 至此我们得到了句向量

['我', '来自', '清华', '大学']
Tensor(shape=[4, 300], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[-0.07075300, -0.00312400,  0.18455200, ...,  0.04748000,
         -0.22542900, -0.42837900],
        [ 0.13855401,  0.08321500, -0.21475700, ..., -0.07017700,
         -0.20189700, -0.03826100],
        [ 0.28273299,  0.00419400, -0.36795101, ...,  0.02538100,
          0.09789900, -0.19698900],
        [ 0.06500300,  0.19419700, -0.33714399, ..., -0.09627100,
          0.06636300, -0.22596300]])
Tensor(shape=[1, 300], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[ 0.41553700,  0.27848199, -0.73529994, -0.39644498, -1.35441804,
          0.14823900,  0.61021996,  0.22136298,  0.01457500, -0.51933700,
         -0.03049497,  0.29500899,  1.21817601,  1.52853608, -0.29614100,
         -0.22853000,  0.56629598,  0.18215299,  0.05563702, -0.14031701,
          0.46038696, -0.39250997, -0.14935900, -0.18032800,  0.09984998,
          0.82393301,  0.19287001,  1.3

# Attension 机制
